In [1]:
from IPython.display import Markdown as md
import math 

In [2]:
V_CMDS = 12
V_CMSYS = 5
CMRR_MIN = 146
VS_SYS = 5
VS_DS = 5
PSRR_MIN = 1111
T1 = 25
T2 = 120
SHUNT_VOS_DRIFT = 0.01e-6

G_err25c = 0.001 # %0.1
R_shunt = 0.1
A_shunt = 0.01
V_shunt = A_shunt * R_shunt 
LSB_Vshunt = 312e-9

## Determine the maximum offset for $V_{SHUNT}$ measurement

$V_{OS_{total}} = V_{OSI} + V_{OS_{CMRR}} + V_{OS_{PSRR}} + V_{OS_{drift}}$

In [3]:
V_OSI = 500e-06
md("Datasheet says $V_{{OSI}}$ = {} V i.e. {} uV".format(V_OSI,V_OSI*1000000))

Datasheet says $V_{OSI}$ = 0.0005 V i.e. 500.0 uV

$V_{OS_{CMRR}} = |V_{CM}-V_{CMDS}| * 10^{-\frac{CMRR_{MIN}}{20dB}} $

In [4]:
V_OS_cmrr = math.fabs(V_CMDS - V_CMSYS)*math.pow(10,-(CMRR_MIN/20))
md("$V_{{OS_{{CMRR}}}}$ = {} V i.e. {} uV".format(V_OS_cmrr,V_OS_cmrr*1000000))

$V_{OS_{CMRR}}$ = 3.5083106353909075e-07 V i.e. 0.35083106353909077 uV

$ V_{OS_{PSRR}} = |VS_{DS}-VS_{SYS}| * PSRR_{MIN} $

In [5]:
V_OS_psrr = math.fabs(VS_SYS - VS_DS)*PSRR_MIN
md("$V_{{OS_{{PSRR}}}}$ = {} V i.e. {} uV".format(V_OS_psrr,V_OS_psrr*1000000))

$V_{OS_{PSRR}}$ = 0.0 V i.e. 0.0 uV

$ V_{OS_{drift}} = |\Delta T| * \frac{\delta V_{OS}}{\delta T} $

In [6]:
V_OS_drift = math.fabs(T1-T2) * SHUNT_VOS_DRIFT
md("$V_{{OS_{{drift}}}}$ = {} V i.e. {} uV".format(V_OS_drift,V_OS_drift*1000000))

$V_{OS_{drift}}$ = 9.5e-07 V i.e. 0.95 uV

In [7]:
V_OS_total = V_OSI+V_OS_cmrr+V_OS_psrr+V_OS_drift
md("$V_{{OS_{{total}}}}$ = {} V i.e. {} uV".format(V_OS_total,V_OS_total*1000000))

$V_{OS_{total}}$ = 0.000501300831063539 V i.e. 501.300831063539 uV

## Error

$ e = \sqrt{ (V_{OSI} * V_{shunt} + | V_{CMDS} - V_{CMSYS}| * V_{OS_{CMRR}} + |VS_{SYS} - VS_{DS}| * V_{OS_{PSRR}})^2 + e_{GN}^2 } $

In [8]:
print("V_shunt = ", V_shunt)
e_vos = V_OSI * V_shunt
print("e_vos=",e_vos*100)

cmrr = math.pow(10, -84/20)
print("cmrr=",cmrr)
e_cmrr = (math.fabs(V_CMDS - V_CMSYS)*cmrr)/V_shunt
print("e_cmrr=",e_cmrr*100)

eGain = 0.01

e2 = math.pow(e_vos*100,2) + math.pow(e_cmrr*100,2) + math.pow(eGain*100,2)
print("e2=", e2)
print("sqrt(e2)=",math.sqrt(e2))
print("----")
e_test = math.pow(e_vos + e_cmrr,2) + math.pow(eGain,2)
print("e_test=",e_test*100)
print("sqrt(e_test)=",math.sqrt(e_test)*100)

#e = math.sqrt( math.pow(V_OSI * V_shunt + (math.fabs(V_CMDS - V_CMSYS)*cmrr)/V_shunt + math.fabs(VS_SYS - VS_DS)*V_OS_psrr,2)+math.pow(G_err25c,2))
#print(e)
#print(e*100)
#print((math.fabs(V_CMDS - V_CMSYS)*cmrr)/V_shunt)



V_shunt =  0.001
e_vos= 4.9999999999999996e-05
cmrr= 6.309573444801929e-05
e_cmrr= 44.167014113613504
e2= 1951.7251357146345
sqrt(e2)= 44.1783333288461
----
e_test= 19.517295524160456
sqrt(e_test)= 44.178383316007


## Determine the maximum ADC gain error for $V_{SHUNT}$ measurement

$G_{err\_drift} = |\Delta T | * 20ppm $

In [9]:
G_err_drift = math.fabs(T1-T2) * 20e-6
md("$G_{{err\_drift}}$ = {}".format(G_err_drift))

$G_{err\_drift}$ = 0.0019000000000000002

In [10]:
G_err_total = G_err25c + G_err_drift
md("$G_{{err\_total}}$ = {}".format(G_err_total))

$G_{err\_total}$ = 0.0029000000000000002

## Convert to digital codes and calculate total error at 1 mA load

$ G_{IDEAL} = \frac{1}{LSB_{V_{SHUNT}}} $

In [11]:
G_IDEAL = 1/LSB_Vshunt
md("$G_{{IDEAL}}$ = {}".format(G_IDEAL))

$G_{IDEAL}$ = 3205128.2051282055